In [56]:
import pandas as pd
import re
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [57]:
times = pd.read_csv('timesData.csv', sep= ',')
shangai = pd.read_csv('shanghaiData.csv')
cwur = pd.read_csv('cwurData.csv')
attainment = pd.read_csv('educational_attainment_supplementary_data.csv')

In [58]:
times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [59]:
cwur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


In [60]:
shangai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4897 entries, 0 to 4896
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       4897 non-null   object 
 1   university_name  4896 non-null   object 
 2   national_rank    4896 non-null   object 
 3   total_score      1101 non-null   float64
 4   alumni           4896 non-null   float64
 5   award            4895 non-null   float64
 6   hici             4895 non-null   float64
 7   ns               4875 non-null   float64
 8   pub              4895 non-null   float64
 9   pcp              4895 non-null   float64
 10  year             4897 non-null   int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 421.0+ KB


## Uniformiamo i Dataframe

In [61]:
cwur=cwur.rename(columns={"institution":"university_name"})

In [62]:
shangai['university_name'] = shangai['university_name'].astype(str) 
#trasformazione di tipo perchè l'argomento di fuzz.partial_ratio deve essere una stringa

Le funzioni del pacchetto fuzzywuzzy calcolano un “similarity score” basato sulla distanza di Levenshtein tra due stringhe A e B, ovvero il numero minimo di modifiche elementari (cancellazione, sostituzione o inserimento di un carattere) che consentono di trasformare la A nella B

In [63]:
#partial_ratio di fuzzywuzzy esegue matching di sottostringhe e ci permette di ricavare il loro similarity score
def match_names(name, list_names, min_score=0):   
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.partial_ratio(name, x)
        if (score > min_score) & (score > max_score) & (x !=name):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [64]:
cw_uni = list(cwur['university_name'].unique())
sh_uni = list(shangai['university_name'].unique())
tm_uni = list(times['university_name'].unique())

In [66]:
# il ciclo for crea una lista, successivamente convertita in dizionario le cui chiave è la stringa che andremo a sostituire, 
# mentre il valore è la stringa con cui lo sostituiremo
names = []
for x in tm_uni:
    match = match_names(x, tm_uni, 97)
    if match[1] >= 97:
        name = set([x, match[0]])
        names.append(name)
name_dict_tm = dict(names)
name_dict_tm


{'University of Hong Kong': 'Chinese University of Hong Kong',
 'École Polytechnique Fédérale de Lausanne': 'École Polytechnique',
 'École Normale Supérieure': 'École Normale Supérieure de Lyon',
 'University of Wisconsin': 'University of Wisconsin-Madison',
 'New York University': 'York University',
 'National Taiwan University': 'National Taiwan University of Science and Technology (Taiwan Tech)',
 'University of Barcelona': 'Autonomous University of Barcelona',
 'University of Vienna': 'Medical University of Vienna',
 'University of South Carolina': 'Medical University of South Carolina',
 'Queen’s University Belfast': 'Queen’s University',
 'City University of Hong Kong': 'University of Hong Kong',
 'University of Milan': 'University of Milan-Bicocca',
 'University of Kentucky': 'University of Kent',
 'Polytechnic University of Milan': 'University of Milan',
 'University of Valencia': 'Polytechnic University of Valencia',
 'New University of Lisbon': 'University of Lisbon',
 'Unive

In [67]:
names = []
for x in sh_uni:
    match = match_names(x, tm_uni, 97)
    if match[1] >= 97:
        name = set([x, match[0]])
        names.append(name)
name_dict_sh2tm = dict(names)
name_dict_sh2tm

{'Massachusetts Institute of Technology (MIT)': 'Massachusetts Institute of Technology',
 'University of Wisconsin - Madison': 'University of Wisconsin',
 'Johns Hopkins University': 'The Johns Hopkins University',
 'University of Tokyo': 'The University of Tokyo',
 'University of Michigan': 'University of Michigan - Ann Arbor',
 'ETH Zurich – Swiss Federal Institute of Technology Zurich': 'Swiss Federal Institute of Technology Zurich',
 'New York University': 'York University',
 'University of Minnesota': 'University of Minnesota, Twin Cities',
 'University of Texas at Austin': 'The University of Texas at Austin',
 'Pennsylvania State University - University Park': 'Pennsylvania State University',
 'Rutgers, the State University of New Jersey': 'Rutgers, The State University of New Jersey - New Brunswick',
 'The University of Edinburgh': 'University of Edinburgh',
 'University of Munich': 'Technical University of Munich',
 'University of Manchester': 'The University of Manchester',
 '

In [68]:
names = []
for x in cw_uni:
    match = match_names(x, tm_uni, 97)
    if match[1] >= 97:
        name = set([x, match[0]])
        names.append(name)
name_dict_cw2tm = dict(names)
name_dict_cw2tm

{'New York University': 'York University',
 'University of Wisconsin–Madison': 'University of Wisconsin',
 'University of Illinois at Urbana-Champaign': 'University of Illinois at Urbana–Champaign',
 'University of Michigan': 'University of Michigan, Ann Arbor',
 'University of Washington - Seattle': 'University of Washington',
 'Purdue University, West Lafayette': 'Purdue University',
 'University of Minnesota': 'University of Minnesota, Twin Cities',
 'Ohio State University, Columbus': 'Ohio State University',
 'École Polytechnique Fédérale de Lausanne': 'École Polytechnique',
 'Pennsylvania State University': 'Pennsylvania State University, University Park',
 'University of Pittsburgh': 'University of Pittsburgh - Pittsburgh Campus',
 'Texas A&M University': 'Texas A&M University, College Station',
 'The University of Queensland': 'University of Queensland',
 'Indiana University - Bloomington': 'Indiana University',
 'National Taiwan University': 'National Taiwan University of Scien

In [69]:
cwur['university_name'] = cwur['university_name'].replace(name_dict_cw2tm)
cwur

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [70]:
shangai['university_name'] = shangai['university_name'].replace(name_dict_sh2tm)
shangai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005
...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4893,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4894,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4895,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015


In [71]:
times['income']= times['income'].apply(pd.to_numeric, errors= 'coerce')
#trasformazione di income per calcolare la differenza finale del punto 2 
#poichè income è considerato come stringa e non un numero
#tutto quello che non viene considerato come numero e non si riesce a convertire viene considerato come Nan

In [72]:
times['female_male_ratio'] = times['female_male_ratio'].astype(str)

In [73]:
times['num_students'] =times['num_students'].str.replace(',',"").astype(float)
#poichè le migliaia erano divise con la virgola

In [74]:
cwur['world_rank']= cwur['world_rank'].astype(str)

## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [75]:
df1_new = times.iloc[times.groupby('university_name').idxmax()['year']]
df1_new

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016


In [76]:
df1_old = times.iloc[times.groupby('university_name').idxmin()['year']]
df1_old

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,17422.0,15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,16099.0,24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,9252.0,19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,2400.0,7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,2218.0,8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,2429.0,4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,9666.0,10.5,54%,27 : 73,2011


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [77]:
df_new = df1_new[['university_name','year','income']]  #ricaviamo solo le colonne che ci interessano
df_new

,university_name,year,income
2405,AGH University of Science and Technology,2016,NaN
2003,Aalborg University,2016,43.7
2056,Aalto University,2016,61.6
1908,Aarhus University,2016,68.3
2105,Aberystwyth University,2016,31.3
...,...,...,...
1856,École Normale Supérieure,2016,37.1
2013,École Normale Supérieure de Lyon,2016,31.7
1904,École Polytechnique,2016,82.3
1833,École Polytechnique Fédérale de Lausanne,2016,65.4


In [78]:
df_old = df1_old[['university_name','year','income']]
df_old

,university_name,year,income
2405,AGH University of Science and Technology,2016,NaN
501,Aalborg University,2012,36.4
502,Aalto University,2012,61.9
166,Aarhus University,2011,61.5
476,Aberystwyth University,2012,35.5
...,...,...,...
41,École Normale Supérieure,2011,30.7
99,École Normale Supérieure de Lyon,2011,26.1
38,École Polytechnique,2011,NaN
47,École Polytechnique Fédérale de Lausanne,2011,38.0


In [79]:
df2 = pd.merge(df_new, df_old, on= 'university_name', suffixes=('_new','_old'))
df2

,university_name,year_new,income_new,year_old,income_old
0,AGH University of Science and Technology,2016,NaN,2016,NaN
1,Aalborg University,2016,43.7,2012,36.4
2,Aalto University,2016,61.6,2012,61.9
3,Aarhus University,2016,68.3,2011,61.5
4,Aberystwyth University,2016,31.3,2012,35.5
...,...,...,...,...,...
813,École Normale Supérieure,2016,37.1,2011,30.7
814,École Normale Supérieure de Lyon,2016,31.7,2011,26.1
815,École Polytechnique,2016,82.3,2011,NaN
816,École Polytechnique Fédérale de Lausanne,2016,65.4,2011,38.0


In [80]:
df2['improvement'] = df2['income_new'] - df2['income_old']
df2
#abbiamo deciso di non applicare nessuna trasformazione ai NaN poiché non sappiamo con certezza il valore delle celle.
#sostituirli con gli zeri potrebbe risultare una forzatura

,university_name,year_new,income_new,year_old,income_old,improvement
0,AGH University of Science and Technology,2016,NaN,2016,NaN,NaN
1,Aalborg University,2016,43.7,2012,36.4,7.3
2,Aalto University,2016,61.6,2012,61.9,-0.3
3,Aarhus University,2016,68.3,2011,61.5,6.8
4,Aberystwyth University,2016,31.3,2012,35.5,-4.2
...,...,...,...,...,...,...
813,École Normale Supérieure,2016,37.1,2011,30.7,6.4
814,École Normale Supérieure de Lyon,2016,31.7,2011,26.1,5.6
815,École Polytechnique,2016,82.3,2011,NaN,NaN
816,École Polytechnique Fédérale de Lausanne,2016,65.4,2011,38.0,27.4


## 3. Find the university with the largest increase computed in the previous point

In [81]:
df3 = df2[df2['improvement']== df2['improvement'].max()] 
df3

,university_name,year_new,income_new,year_old,income_old,improvement
428,TU Dresden,2016,99.7,2012,31.9,67.8


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [82]:
df1_new.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016


In [83]:
df4_tm= df1_new[['world_rank','university_name','year']]
df4_tm['world_rank_in']= df4_tm['world_rank'].str.extract('(\d+)').astype(int)
df4_tm['world_rank_fi']= df4_tm['world_rank'].str.extract('(\d+$)').astype(int)
df4_tm['rank_mean']=(df4_tm['world_rank_in']+df4_tm['world_rank_fi'])//2
df4_tm 

<ipython-input-83-28062b686553>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_tm['world_rank_in']= df4_tm['world_rank'].str.extract('(\d+)').astype(int)
<ipython-input-83-28062b686553>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_tm['world_rank_fi']= df4_tm['world_rank'].str.extract('(\d+$)').astype(int)
<ipython-input-83-28062b686553>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,world_rank,university_name,year,world_rank_in,world_rank_fi,rank_mean
2405,601-800,AGH University of Science and Technology,2016,601,800,700
2003,201-250,Aalborg University,2016,201,250,225
2056,251-300,Aalto University,2016,251,300,275
1908,=106,Aarhus University,2016,106,106,106
2105,301-350,Aberystwyth University,2016,301,350,325
...,...,...,...,...,...,...
1856,54,École Normale Supérieure,2016,54,54,54
2013,201-250,École Normale Supérieure de Lyon,2016,201,250,225
1904,=101,École Polytechnique,2016,101,101,101
1833,31,École Polytechnique Fédérale de Lausanne,2016,31,31,31


In [84]:
#time= times.iloc[times.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
#time['world_rank_in']= time['world_rank'].str.extract('(\d+)').astype(int)
#time['world_rank_fi']= time['world_rank'].str.extract('(\d+$)').astype(int)
#time['rank_mean']=(time['world_rank_in']+time['world_rank_fi'])//2
#time

# in questo caso non esce l'avviso di errore quindi l'avviso non si applica al nostro caso

In [85]:
df4_sh= shangai.iloc[shangai.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
df4_sh['world_rank_in']= df4_sh['world_rank'].str.extract('(\d+)').astype(int)
df4_sh['world_rank_fi']= df4_sh['world_rank'].str.extract('(\d+$)').astype(int)
df4_sh['rank_mean']=(df4_sh['world_rank_in']+df4_sh['world_rank_fi'])//2
df4_sh

,world_rank,university_name,year,world_rank_in,world_rank_fi,rank_mean
4697,301-400,Aalborg University,2015,301,400,350
4797,401-500,Aalto University,2015,401,500,450
4469,73,Aarhus University,2015,73,73,73
4497,101-150,Aix Marseille University,2015,101,150,125
3115,102-150,Aix-Marseille University,2011,102,150,126
...,...,...,...,...,...,...
3512,401-500,Yamaguchi University,2011,401,500,450
4695,201-300,Yeshiva University,2015,201,300,250
4696,201-300,Yonsei University,2015,201,300,250
4423,27,York University,2015,27,27,27


In [86]:
df4_cw= cwur.iloc[cwur.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
df4_cw['world_rank_in']= df4_cw['world_rank'].str.extract('(\d+)').astype(int)
df4_cw['world_rank_fi']= df4_cw['world_rank'].str.extract('(\d+$)').astype(int)
df4_cw['rank_mean']=(df4_cw['world_rank_in']+df4_cw['world_rank_fi'])//2
df4_cw

,world_rank,university_name,year,world_rank_in,world_rank_fi,rank_mean
1981,782,AGH University of Science and Technology,2015,782,782,782
1764,565,Aalborg University,2015,565,565,565
1620,421,Aalto University,2015,421,421,421
1321,122,Aarhus University,2015,122,122,122
2013,814,Aberystwyth University,2015,814,814,814
...,...,...,...,...,...,...
2080,881,École centrale de Lyon,2015,881,881,881
1236,37,École normale supérieure - Paris,2015,37,37,37
1920,721,École normale supérieure de Cachan,2015,721,721,721
1670,471,École normale supérieure de Lyon,2015,471,471,471


In [87]:
df4_tm_sh = pd.merge(df4_tm[['university_name','rank_mean']], df4_sh[['university_name','rank_mean']], on='university_name',suffixes=('_time','_shang'))
df4 = pd.merge(df4_tm_sh[['university_name','rank_mean_time','rank_mean_shang']], df4_cw[['university_name','rank_mean']], on='university_name')
df4=df4.rename(columns={'rank_mean':"rank_mean_cwur"})
df4

,university_name,rank_mean_time,rank_mean_shang,rank_mean_cwur
0,Aalborg University,225,350,565
1,Aalto University,275,450,421
2,Aarhus University,106,73,122
3,Aix-Marseille University,275,126,206
4,Aristotle University of Thessaloniki,700,450,459
...,...,...,...,...
381,Yale University,12,11,11
382,Yeshiva University,164,250,171
383,Yonsei University,325,250,98
384,York University,325,27,18


In [88]:
def max_diff(riga):
    sh = riga['rank_mean_shang']
    cw = riga['rank_mean_cwur']
    tm = riga['rank_mean_time']
    m = min(sh, cw, tm)
    M = max(sh, cw, tm)
    return (M - m)

In [89]:
df4['diff_rank'] = df4.apply(max_diff, axis=1)
df4

,university_name,rank_mean_time,rank_mean_shang,rank_mean_cwur,diff_rank
0,Aalborg University,225,350,565,340
1,Aalto University,275,450,421,175
2,Aarhus University,106,73,122,49
3,Aix-Marseille University,275,126,206,149
4,Aristotle University of Thessaloniki,700,450,459,250
...,...,...,...,...,...
381,Yale University,12,11,11,1
382,Yeshiva University,164,250,171,86
383,Yonsei University,325,250,98,227
384,York University,325,27,18,307


## 5. Consider only the most recent data point of the "times" dataset. Compute the number of male and female students for each country.

In [90]:
times['female_%']= times['female_male_ratio'].str.extract('(\d+)').astype(float)  #creiamo una nuova colonna per tutto il 
#dataframe poichè i dati ci serviranno anche per le operazioni successive
times

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,nan,2011,NaN
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011,33.0
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011,37.0
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011,42.0
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,21958.0,15.3,3%,48 : 52,2016,48.0
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,31268.0,28.7,2%,36 : 64,2016,36.0
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,4122.0,3.7,3%,nan,2016,NaN
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,10117.0,12.1,8%,28 : 72,2016,28.0


In [91]:
df_f = times.iloc[times.groupby('university_name').idxmax()['year']][['university_name','country','year','num_students','female_%']]
df_f

,university_name,country,year,num_students,female_%
2405,AGH University of Science and Technology,Poland,2016,35569.0,NaN
2003,Aalborg University,Denmark,2016,17422.0,48.0
2056,Aalto University,Finland,2016,16099.0,32.0
1908,Aarhus University,Denmark,2016,23895.0,54.0
2105,Aberystwyth University,United Kingdom,2016,9252.0,48.0
...,...,...,...,...,...
1856,École Normale Supérieure,France,2016,2400.0,46.0
2013,École Normale Supérieure de Lyon,France,2016,2218.0,49.0
1904,École Polytechnique,France,2016,2429.0,18.0
1833,École Polytechnique Fédérale de Lausanne,Switzerland,2016,9666.0,27.0


In [92]:
df_f['female_stud'] = (df_f['num_students'] *(df_f['female_%']/100)).round(0)
df_f

,university_name,country,year,num_students,female_%,female_stud
2405,AGH University of Science and Technology,Poland,2016,35569.0,NaN,NaN
2003,Aalborg University,Denmark,2016,17422.0,48.0,8363.0
2056,Aalto University,Finland,2016,16099.0,32.0,5152.0
1908,Aarhus University,Denmark,2016,23895.0,54.0,12903.0
2105,Aberystwyth University,United Kingdom,2016,9252.0,48.0,4441.0
...,...,...,...,...,...,...
1856,École Normale Supérieure,France,2016,2400.0,46.0,1104.0
2013,École Normale Supérieure de Lyon,France,2016,2218.0,49.0,1087.0
1904,École Polytechnique,France,2016,2429.0,18.0,437.0
1833,École Polytechnique Fédérale de Lausanne,Switzerland,2016,9666.0,27.0,2610.0


In [93]:
df5 = df_f.groupby('country')[['num_students','female_stud']].sum()
df5

,num_students,female_stud
country,,
Argentina,108373.0,67191.0
Australia,743627.0,391736.0
Austria,134477.0,68364.0
Bangladesh,62716.0,21323.0
Belarus,29303.0,20219.0
...,...,...
Unisted States of America,0.0,0.0
United Arab Emirates,14447.0,9516.0
United Kingdom,1324842.0,711815.0


In [94]:
df5['male_stud'] = df5['num_students'] - df5['female_stud']
df5

,num_students,female_stud,male_stud
country,,,
Argentina,108373.0,67191.0,41182.0
Australia,743627.0,391736.0,351891.0
Austria,134477.0,68364.0,66113.0
Bangladesh,62716.0,21323.0,41393.0
Belarus,29303.0,20219.0,9084.0
...,...,...,...
Unisted States of America,0.0,0.0,0.0
United Arab Emirates,14447.0,9516.0,4931.0
United Kingdom,1324842.0,711815.0,613027.0


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [95]:
times['ratio'] = (times['female_%']/100)/(1-times['female_%']/100)
times

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,ratio
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,nan,2011,NaN,NaN
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011,33.0,0.492537
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011,37.0,0.587302
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011,42.0,0.724138
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011,45.0,0.818182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,21958.0,15.3,3%,48 : 52,2016,48.0,0.923077
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,31268.0,28.7,2%,36 : 64,2016,36.0,0.562500
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,4122.0,3.7,3%,nan,2016,NaN,NaN
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,10117.0,12.1,8%,28 : 72,2016,28.0,0.388889


In [96]:
times[times['ratio']== np.inf]
#siccome l'output della riga precedente poteva essere infinito abbiamo deciso di sostituirlo con NaN, poichè il valore 
#infinito era privo di significato in questo caso.

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,ratio
1760,351-400,Ewha Womans University,South Korea,23.5,36.0,18.0,57.6,61.3,-,17625.0,14.4,6%,100 : 0,2015,100.0,inf
2227,401-500,Ewha Womans University,South Korea,23.9,34.3,18.3,50.4,57.6,-,17625.0,14.4,6%,100 : 0,2016,100.0,inf


In [97]:
times['ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [98]:
mean_ratio= times['ratio'].mean()
mean_ratio

1.0684463755952165

In [99]:
df6=times[times['ratio']<=mean_ratio]
df6['university_name'].unique

<bound method Series.unique of 1          California Institute of Technology
2       Massachusetts Institute of Technology
3                         Stanford University
4                        Princeton University
5                     University of Cambridge
                        ...                  
2597                        Xidian University
2598                      Yeungnam University
2599              Yıldız Technical University
2601             Yokohama National University
2602                       Yuan Ze University
Name: university_name, Length: 1078, dtype: object>

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).


In [100]:
df_by_country = times.groupby('country', as_index=False).sum()[['country','num_students']]
df_by_country

,country,num_students
0,Argentina,108373.0
1,Australia,3082024.0
2,Austria,668005.0
3,Bangladesh,62716.0
4,Belarus,29303.0
...,...,...
67,Unisted States of America,0.0
68,United Arab Emirates,14447.0
69,United Kingdom,4926093.0
70,United States of America,15030134.0


In [101]:
df7= pd.merge(df6, df_by_country, on = 'country')
df7= df7.rename(columns={"num_students_y":"tot_students_ country"})
df7

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students_x,student_staff_ratio,international_students,female_male_ratio,year,female_%,ratio,tot_students_ country
0,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011,33.0,0.492537,15030134.0
1,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011,37.0,0.587302,15030134.0
2,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011,42.0,0.724138,15030134.0
3,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011,45.0,0.818182,15030134.0
4,8,"University of California, Berkeley",United States of America,84.2,39.6,99.3,97.8,NaN,91.1,36186.0,16.4,15%,50 : 50,2011,50.0,1.000000,15030134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,601-800,University of Nairobi,Kenya,13.7,45.5,9.0,19.5,28.0,-,72676.0,47.0,1%,40 : 60,2016,40.0,0.666667,72676.0
1074,601-800,National University of Sciences and Technology,Pakistan,21.4,31.3,7.0,35.0,31.3,-,9808.0,7.8,3%,27 : 73,2016,27.0,0.369863,22496.0
1075,601-800,Slovak University of Technology in Bratislava,Slovakia,23.3,30.6,8.6,24.9,33.2,-,16400.0,15.3,2%,31 : 69,2016,31.0,0.449275,37279.0
1076,601-800,Sultan Qaboos University,Oman,21.3,71.5,10.6,16.0,37.6,-,7089.0,9.0,3%,50 : 50,2016,50.0,1.000000,7089.0


In [102]:
df7['fraction']= df7['num_students_x']/df7['tot_students_ country']*100
df7[['university_name','country','num_students_x','tot_students_ country','fraction']]

,university_name,country,num_students_x,tot_students_ country,fraction
0,California Institute of Technology,United States of America,2243.0,15030134.0,0.014923
1,Massachusetts Institute of Technology,United States of America,11074.0,15030134.0,0.073679
2,Stanford University,United States of America,15596.0,15030134.0,0.103765
3,Princeton University,United States of America,7929.0,15030134.0,0.052754
4,"University of California, Berkeley",United States of America,36186.0,15030134.0,0.240756
...,...,...,...,...,...
1073,University of Nairobi,Kenya,72676.0,72676.0,100.000000
1074,National University of Sciences and Technology,Pakistan,9808.0,22496.0,43.598862
1075,Slovak University of Technology in Bratislava,Slovakia,16400.0,37279.0,43.992596
1076,Sultan Qaboos University,Oman,7089.0,7089.0,100.000000


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [103]:
attainment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79055 entries, 0 to 79054
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_name  79052 non-null  object 
 1   series_name   79050 non-null  object 
 2   1985          51843 non-null  float64
 3   1986          12 non-null     float64
 4   1987          3 non-null      float64
 5   1990          51846 non-null  float64
 6   1991          1 non-null      float64
 7   1992          3 non-null      float64
 8   1993          1 non-null      float64
 9   1995          51884 non-null  float64
 10  1996          3 non-null      float64
 11  1997          0 non-null      float64
 12  1998          47 non-null     float64
 13  1999          9 non-null      float64
 14  2000          51970 non-null  float64
 15  2001          225 non-null    float64
 16  2002          184 non-null    float64
 17  2003          55 non-null     float64
 18  2004          137 non-null

In [104]:
attainment

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df8 = attainment.dropna(subset= ['country_name', 'series_name'])

In [106]:
df8['country_name'].isnull().sum()

0

In [107]:
df8

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [108]:
#ricaviamo una lista delle etichette delle colonne che poi saranno valori di celle.
years = list(attainment.columns)
del years[0:2]
years

['1985',
 '1986',
 '1987',
 '1990',
 '1991',
 '1992',
 '1993',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2015']

In [109]:
df9 = pd.melt(df8, id_vars = ['country_name','series_name'], value_vars = years, var_name = 'year').dropna(subset = ['value'])
df9
# melt è una funzione per trasformare il dataframe da un formato ampio in un formato lungo.
#id_vars sono le colonne identificative che rimangono fisse, mentre le value_vars sono le righe che vengono trasposte.
#var_name è l’etichetta della nuova colonna che si genera.

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


In [110]:
# indice = ['1985', '1986', '1987', '1990', '1991',
#        '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
#        '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
#        '2011', '2012', '2013', '2015']

# def func(riga):
#     country = riga['country_name']
#     series = riga['series_name']
#     for anno in indice:
#         if not pd.isna(riga[anno]):
#             el = riga[anno]
#             dfx = pd.Series([country, series, anno, el], index = ['country_name', 'series_name', 'year', 'value'])
#             df_lenght = len(df9)
#             df9.loc[df_lenght] = dfx
#     return df9
#df8.apply(func, axis = 1)

## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [111]:
rank_cw = pd.DataFrame(cwur.groupby('university_name').count()['year'])
rank_cw['cwur'] = 1
rank_cw.drop('year',axis=1, inplace= True)
rank_cw
#utilizziamo il groupby per ricavare 3 df (uno per ogni ranking) in cui ogni università appare una sola volta.
#Il count() è indispensabile per ricavarlo, ma l’informazione che dà non serve, infatti la eliminiamo.
#per ogni ranking inoltre aggiungiamo una colonna di “1” per associare ogni università al ranking di provenienza e 
#concateniamo i 3 df.

,cwur
university_name,
AGH University of Science and Technology,1
Aalborg University,1
Aalto University,1
Aarhus University,1
Aberystwyth University,1
...,...
École centrale de Lyon,1
École normale supérieure - Paris,1
École normale supérieure de Cachan,1


In [112]:
rank_sh = pd.DataFrame(shangai.groupby('university_name').count()['year'])
rank_sh['shangai'] = 1
rank_sh.drop('year',axis=1, inplace= True)
rank_sh

,shangai
university_name,
Aalborg University,1
Aalto University,1
Aarhus University,1
Aix Marseille University,1
Aix-Marseille University,1
...,...
Yamaguchi University,1
Yeshiva University,1
Yonsei University,1


In [113]:
rank_tm = pd.DataFrame(times.groupby('university_name').count()['year'])
rank_tm['times'] = 1
rank_tm.drop('year',axis=1, inplace= True)
rank_tm

,times
university_name,
AGH University of Science and Technology,1
Aalborg University,1
Aalto University,1
Aarhus University,1
Aberystwyth University,1
...,...
École Normale Supérieure,1
École Normale Supérieure de Lyon,1
École Polytechnique,1


In [114]:
df10 = pd.concat([rank_cw, rank_sh, rank_tm],axis = 1, sort = True)
df10 = df10.replace(np.nan, 0)
df10

,cwur,shangai,times
AGH University of Science and Technology,1.0,0.0,1.0
Aalborg University,1.0,1.0,1.0
Aalto University,1.0,1.0,1.0
Aarhus University,1.0,1.0,1.0
Aberystwyth University,1.0,0.0,1.0
...,...,...,...
École centrale de Lyon,1.0,0.0,0.0
École normale supérieure - Paris,1.0,0.0,0.0
École normale supérieure de Cachan,1.0,0.0,0.0
École normale supérieure de Lyon,1.0,0.0,0.0


In [115]:
#Troviamo quante volte una determinata università compare nei ranking sommando i valori delle colonne del df risultato
df10['num_ranks'] = df10['cwur'].astype(int) + df10['shangai'].astype(int) + df10['times'].astype(int)
df10

,cwur,shangai,times,num_ranks
AGH University of Science and Technology,1.0,0.0,1.0,2
Aalborg University,1.0,1.0,1.0,3
Aalto University,1.0,1.0,1.0,3
Aarhus University,1.0,1.0,1.0,3
Aberystwyth University,1.0,0.0,1.0,2
...,...,...,...,...
École centrale de Lyon,1.0,0.0,0.0,1
École normale supérieure - Paris,1.0,0.0,0.0,1
École normale supérieure de Cachan,1.0,0.0,0.0,1
École normale supérieure de Lyon,1.0,0.0,0.0,1


## 11. In the times ranking, compute the number of times each university appears

In [116]:
#generiamo un nuovo Dataframe con la colonna "count" 
df11=pd.DataFrame(columns=['count'])
df11['count']=times.groupby('university_name', as_index=True).size()
df11
#size() calcola il numero di elementi nella group by

,count
university_name,
AGH University of Science and Technology,1
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
...,...
École Normale Supérieure,6
École Normale Supérieure de Lyon,6
École Polytechnique,6


## 12. Find the universities that appear at most twice in the times ranking

In [117]:
df12=df11[df11['count']<=2]
df12

,count
university_name,
AGH University of Science and Technology,1
Adam Mickiewicz University,1
Aix-Marseille University,1
Ajou University,1
Alexandru Ioan Cuza University,1
...,...
Yokohama City University,1
Yokohama National University,1
Yuan Ze University,2


## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).


In [118]:
tm_sh=pd.merge(times[['world_rank', 'university_name','year']], shangai[['world_rank', 'university_name','year']], on=['world_rank','university_name','year'])
tm_sh
#come chiave della merge abbiamo utilizzato 3 variabili

,world_rank,university_name,year
0,1,Harvard University,2011
1,3,Massachusetts Institute of Technology,2011
2,2,Stanford University,2012
3,9,University of Chicago,2012
4,11,Yale University,2012
5,96,Michigan State University,2012
6,2,Stanford University,2013
7,11,Yale University,2013
8,15,University of Pennsylvania,2013
9,35,"University of California, Santa Barbara",2013


In [119]:
df13= pd.merge(tm_sh,cwur[['world_rank','university_name','year']],on= ['world_rank','university_name','year'])
df13

,world_rank,university_name,year
0,2,Stanford University,2013
